# KNN 
## Model

In [6]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold,train_test_split,StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.utils import class_weight
from imblearn.over_sampling import RandomOverSampler
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score

In [2]:
# Read CSV file

data=pd.read_csv('preprocessed_UK_Accidents_2009_updated.csv',index_col='accident_index')
data=data.drop('seasons_ranges',axis=1)
x = data.drop('accident_severity', axis=1) 
y = data['accident_severity']

In [3]:
# Scalling data
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

In [16]:
# Applying PCA

#KNN is a distance-based algorithm that relies on the similarity between data points. 
#As the number of dimensions in the data increases, the distance between points becomes less informative and can lead to 
#the "curse of dimensionality". PCA can reduce the dimensionality of the data while retaining most of the information, 
#making the KNN model more efficient and accurate.

## choosing n_components using elbow method and explained_variance_ratio_

priciple_component_analayzer = PCA(n_components=15) # based on domain knowledge from feature extraction notebook
x_pca = priciple_component_analayzer.fit_transform(x_scaled)

In [17]:
## Data Imbalance
class_counts = y.value_counts()
class_counts

1    134714
2     21475
3      2003
Name: accident_severity, dtype: int64

In [7]:
## As shown above the data suffer from sever data imbalance
## Solutions:
## 1-class weights (not available in KNN)
## 2- Resampling (random resampling will be used in our case )

In [18]:
## Over sampling due to data imbalance 
oversampler = RandomOverSampler(random_state=10)
x_resampled, y_resampled = oversampler.fit_resample(x_pca, y)
x=pd.DataFrame(x_resampled)
y=y_resampled

In [19]:
## Train/Test split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=10,stratify=y)## stratify=y used to have the same ration of classes in each split


In [55]:
## Evaluation Metrics used: 1-accuracy 2-precision 3-recall 4-f1
## But we will be mailnly focusing on recall and f1 as they are more representative to our application, 
## where true positives and false negatives are important 

In [12]:
# Search grid for hyperparamter tuning
x = X_train
y = y_train

# create a KNN model
knn = KNeighborsClassifier()

# define the hyperparameters to search over
param_grid = {'n_neighbors': range(3, 100, 10), 'weights': ['uniform', 'distance'], 'metric': ['euclidean', 'manhattan']}

# create custom scoring functions to calculate accuracy, precision, recall, and F1 score with average='macro'
accuracy_scorer = make_scorer(accuracy_score)
precision_scorer = make_scorer(precision_score, average='micro', zero_division=0)
recall_scorer = make_scorer(recall_score, average='micro')
f1_scorer = make_scorer(f1_score, average='micro')

# perform grid search and obtain performance metrics for each combination of hyperparameters
grid_search = GridSearchCV(knn, param_grid=param_grid, cv=StratifiedKFold(n_splits=5, random_state=10, shuffle=True), scoring={'accuracy': accuracy_scorer, 'precision': precision_scorer, 'recall': recall_scorer, 'f1': f1_scorer}, refit='precision',verbose=3)

# fit the grid search to the data
grid_search.fit(x, y)

# print the best hyperparameters and corresponding performance metrics
print("Best hyperparameters:", grid_search.best_params_)
print("Best accuracy:", grid_search.best_score_)
print("Best precision:", grid_search.cv_results_['mean_test_precision'][grid_search.best_index_])
print("Best recall:", grid_search.cv_results_['mean_test_recall'][grid_search.best_index_])
print("Best F1 score:", grid_search.cv_results_['mean_test_f1'][grid_search.best_index_])

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV 1/5] END metric=euclidean, n_neighbors=3, weights=uniform; accuracy: (test=0.879) f1: (test=0.879) precision: (test=0.879) recall: (test=0.879) total time= 1.1min
[CV 2/5] END metric=euclidean, n_neighbors=3, weights=uniform; accuracy: (test=0.878) f1: (test=0.878) precision: (test=0.878) recall: (test=0.878) total time= 1.1min
[CV 3/5] END metric=euclidean, n_neighbors=3, weights=uniform; accuracy: (test=0.878) f1: (test=0.878) precision: (test=0.878) recall: (test=0.878) total time= 1.1min
[CV 4/5] END metric=euclidean, n_neighbors=3, weights=uniform; accuracy: (test=0.877) f1: (test=0.877) precision: (test=0.877) recall: (test=0.877) total time= 1.1min
[CV 5/5] END metric=euclidean, n_neighbors=3, weights=uniform; accuracy: (test=0.876) f1: (test=0.876) precision: (test=0.876) recall: (test=0.876) total time= 1.1min
[CV 1/5] END metric=euclidean, n_neighbors=3, weights=distance; accuracy: (test=0.892) f1: (test=0.892)

[CV 5/5] END metric=euclidean, n_neighbors=43, weights=distance; accuracy: (test=0.830) f1: (test=0.830) precision: (test=0.830) recall: (test=0.830) total time= 4.3min
[CV 1/5] END metric=euclidean, n_neighbors=53, weights=uniform; accuracy: (test=0.649) f1: (test=0.649) precision: (test=0.649) recall: (test=0.649) total time= 5.2min
[CV 2/5] END metric=euclidean, n_neighbors=53, weights=uniform; accuracy: (test=0.646) f1: (test=0.646) precision: (test=0.646) recall: (test=0.646) total time= 5.2min
[CV 3/5] END metric=euclidean, n_neighbors=53, weights=uniform; accuracy: (test=0.650) f1: (test=0.650) precision: (test=0.650) recall: (test=0.650) total time= 5.2min
[CV 4/5] END metric=euclidean, n_neighbors=53, weights=uniform; accuracy: (test=0.647) f1: (test=0.647) precision: (test=0.647) recall: (test=0.647) total time= 5.1min
[CV 5/5] END metric=euclidean, n_neighbors=53, weights=uniform; accuracy: (test=0.647) f1: (test=0.647) precision: (test=0.647) recall: (test=0.647) total time

[CV 4/5] END metric=euclidean, n_neighbors=93, weights=distance; accuracy: (test=0.815) f1: (test=0.815) precision: (test=0.815) recall: (test=0.815) total time= 7.6min
[CV 5/5] END metric=euclidean, n_neighbors=93, weights=distance; accuracy: (test=0.813) f1: (test=0.813) precision: (test=0.813) recall: (test=0.813) total time= 7.6min
[CV 1/5] END metric=manhattan, n_neighbors=3, weights=uniform; accuracy: (test=0.879) f1: (test=0.879) precision: (test=0.879) recall: (test=0.879) total time= 1.9min
[CV 2/5] END metric=manhattan, n_neighbors=3, weights=uniform; accuracy: (test=0.878) f1: (test=0.878) precision: (test=0.878) recall: (test=0.878) total time= 1.9min
[CV 3/5] END metric=manhattan, n_neighbors=3, weights=uniform; accuracy: (test=0.880) f1: (test=0.880) precision: (test=0.880) recall: (test=0.880) total time= 1.9min
[CV 4/5] END metric=manhattan, n_neighbors=3, weights=uniform; accuracy: (test=0.878) f1: (test=0.878) precision: (test=0.878) recall: (test=0.878) total time= 1

[CV 3/5] END metric=manhattan, n_neighbors=43, weights=distance; accuracy: (test=0.832) f1: (test=0.832) precision: (test=0.832) recall: (test=0.832) total time= 6.9min
[CV 4/5] END metric=manhattan, n_neighbors=43, weights=distance; accuracy: (test=0.832) f1: (test=0.832) precision: (test=0.832) recall: (test=0.832) total time= 6.9min
[CV 5/5] END metric=manhattan, n_neighbors=43, weights=distance; accuracy: (test=0.831) f1: (test=0.831) precision: (test=0.831) recall: (test=0.831) total time= 6.9min
[CV 1/5] END metric=manhattan, n_neighbors=53, weights=uniform; accuracy: (test=0.653) f1: (test=0.653) precision: (test=0.653) recall: (test=0.653) total time= 8.4min
[CV 2/5] END metric=manhattan, n_neighbors=53, weights=uniform; accuracy: (test=0.649) f1: (test=0.649) precision: (test=0.649) recall: (test=0.649) total time= 8.4min
[CV 3/5] END metric=manhattan, n_neighbors=53, weights=uniform; accuracy: (test=0.652) f1: (test=0.652) precision: (test=0.652) recall: (test=0.652) total ti

[CV 2/5] END metric=manhattan, n_neighbors=93, weights=distance; accuracy: (test=0.814) f1: (test=0.814) precision: (test=0.814) recall: (test=0.814) total time=11.6min
[CV 3/5] END metric=manhattan, n_neighbors=93, weights=distance; accuracy: (test=0.814) f1: (test=0.814) precision: (test=0.814) recall: (test=0.814) total time=11.8min
[CV 4/5] END metric=manhattan, n_neighbors=93, weights=distance; accuracy: (test=0.814) f1: (test=0.814) precision: (test=0.814) recall: (test=0.814) total time=11.2min
[CV 5/5] END metric=manhattan, n_neighbors=93, weights=distance; accuracy: (test=0.813) f1: (test=0.813) precision: (test=0.813) recall: (test=0.813) total time=11.1min
Best hyperparameters: {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}
Best accuracy: 0.8926117838844899
Best precision: 0.8926117838844899
Best recall: 0.8926117838844899
Best F1 score: 0.8926117838844899


In [ ]:
KNN = KNeighborsClassifier(n_neighbors=3,metric='manhattan',weights='distance')
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = cross_validate(KNN, x, y, cv=skf, scoring='f1_micro')
print(sum(results) / len(results))

In [20]:

KNN = KNeighborsClassifier(n_neighbors=3,metric='manhattan',weights='distance')
# Train the model on the training data
KNN.fit(X_train, y_train)

# Evaluate the performance of the model on the training and testing data
y_train_pred = KNN.predict(X_train)
y_test_pred = KNN.predict(X_test)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

train_f1 = f1_score(y_train, y_train_pred, average='micro')
test_f1 = f1_score(y_test, y_test_pred, average='micro')

train_precision = precision_score(y_train, y_train_pred, average='micro')
test_precision = precision_score(y_test, y_test_pred, average='micro')

train_recall = recall_score(y_train, y_train_pred, average='micro')
test_recall = recall_score(y_test, y_test_pred, average='micro')

# Print the evaluation metrics
print("Training accuracy:", train_accuracy)
print("Testing accuracy:", test_accuracy)

print("Training F1 score:", train_f1)
print("Testing F1 score:", test_f1)

print("Training precision:", train_precision)
print("Testing precision:", test_precision)

print("Training recall:", train_recall)
print("Testing recall:", test_recall)

Training accuracy: 0.9999970889613414
Testing accuracy: 0.9083995909075913
Training F1 score: 0.9999970889613414
Testing F1 score: 0.9083995909075913
Training precision: 0.9999970889613414
Testing precision: 0.9083995909075913
Training recall: 0.9999970889613414
Testing recall: 0.9083995909075913


In [ ]:
## The results show that the model is doing well and not overfitting

In [ ]:
# KNN can be a good option for imbalanced data because it is a non-parametric algorithm that does not make any 
# assumptions, like the assumptions made in NB for example, about the underlying distribution of the data. 
# KNN simply classifies new examples based on the class labels of their nearest neighbors in the training data.

# Moreover, KNN is robust against outliers than other algorithms. 
# Outliers can have a significant impact on the decision boundary of a classifier, and can lead to poor performance 
# on the minority class. However, KNN can be less sensitive to outliers because it simply considers the k nearest neighbors, 
# rather than trying to fit a complex decision boundary.